In [1]:
import csv
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

# Spelling corrector (http://norvig.com/spell-correct.html)

In [2]:
def P(word): 
    "Probability of `word`."
    N = sum(WORDS.values())
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnñopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

# Ciudades y barrios

## Socios activos 2016

In [3]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                        95

In [4]:
%%time

dtypes = {
    'CODBAR': str,
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Fech. Nacim': pd.tslib.Timestamp,
    'Edad': float,
}

perfiles_df = pd.read_csv('socios_activos_2016.csv', header=0, dtype=dtypes, parse_dates=['Fech. Nacim'])
perfiles_df = perfiles_df.fillna('')

CPU times: user 366 ms, sys: 0 ns, total: 366 ms
Wall time: 363 ms


In [5]:
perfiles_df.shape

(50272, 11)

In [6]:
perfiles_df.head()

,CODBAR,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim,Edad
0,88180782,BLAA,,chapinero,bogota,CIN,universidad,,,,
1,88183608,BLAA,sra.,ayacuyo ii,bogota,CEM,pensionado,,,1957-07-30 00:00:00,59
2,88181732,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,,
3,88182347,ARM,sr.,montenegro,montenegro,CEM,empleado,profesional,economía,1963-09-20 00:00:00,53
4,88181284,BLAA,sr.,la soledad,bogota,CEM,,,,,


# Limpiando datos

#### Ciudad

In [7]:
perfiles_df['Ciudad'].nunique()

997

In [8]:
perfiles_df['Ciudad'].value_counts().head()

bogota     23087
tunja       2952
ibague      2264
popayan     1862
neiva       1754
Name: Ciudad, dtype: int64

In [9]:
%%time

sucursal_ciudades = {}

for index, row in perfiles_df.iterrows():
    sucursal = row['Sucursal']
    if sucursal:
        ciudad = row['Ciudad']
        sucursal_ciudades.setdefault(sucursal, defaultdict(int))
        sucursal_ciudades[sucursal][ciudad] += 1

CPU times: user 6.83 s, sys: 0 ns, total: 6.83 s
Wall time: 6.82 s


In [10]:
sucursal_ciudad_frecuencia = {}

for k in sucursal_ciudades:
    sucursal_ciudad_frecuencia[k] = Counter(sucursal_ciudades[k]).most_common(1)[0]

In [11]:
ciudades = Counter(dict(sucursal_ciudad_frecuencia.values()))

In [12]:
ciudades

Counter({'armenia': 454,
         'barranquilla': 228,
         'bogota': 22242,
         'bucaramanga': 344,
         'buenaventura': 317,
         'cali': 247,
         'cartagena': 707,
         'cucuta': 199,
         'florencia': 586,
         'girardot': 899,
         'honda': 334,
         'ibague': 2220,
         'ipiales': 983,
         'leticia': 477,
         'manizales': 788,
         'medellin': 163,
         'monteria': 177,
         'neiva': 1742,
         'pasto': 1674,
         'pereira': 1443,
         'popayan': 1841,
         'quibdo': 445,
         'riohacha': 237,
         'san andres': 658,
         'santa marta': 412,
         'sincelejo': 396,
         'tunja': 2930,
         'valledupar': 628,
         'villavicencio': 269})

In [13]:
WORDS = ciudades

In [14]:
%%time

def ciudad_corrector(x):
    ciudad = x['Ciudad']
    if ciudad:
        if ciudad in ciudades:
            r = ciudad
        else:
            words = re.findall(r'\w+', ciudad)
            candidates = set(w for w in words if w in ciudades)
            if len(candidates) == 1:
                r = candidates.pop()
            else:
                # len(candidates) > 1 no se da
                corrections = [correction(w) for w in words]
                candidates = set(c for c in corrections if c in ciudades)
                if len(candidates) == 1:
                    r = candidates.pop()
                else:
                    # len(candidates) > 1 sólo se da una vez pero las correcciones son incorrectas
                    r = ciudad
    else:
        r = ciudad
    return r

perfiles_df['Ciudad'] = perfiles_df.apply(ciudad_corrector, axis=1)

CPU times: user 6min 46s, sys: 503 ms, total: 6min 47s
Wall time: 6min 47s


In [15]:
perfiles_df['Ciudad'].nunique()

559

#### Barrio

In [16]:
perfiles_df['Barrio'].nunique()

11332

In [73]:
perfiles_df['Barrio'].value_counts().head()

centro       1153
chapinero     429
              361
palermo       265
castilla      240
Name: Barrio, dtype: int64

In [91]:
ciudad_barrios = {}

if la ciudad se repite mas de una vez..

for ciudad, barrios in perfiles_df.groupby('Ciudad')['Barrio']:
    ciudad = ciudad.strip()
    if ciudad and barrios.count() > 1:
        ciudad_barrios.setdefault(ciudad, {})
        for b, n in barrios.value_counts().iteritems():
            if b and n > 1:
                ciudad_barrios[ciudad][b] = n

In [92]:
barrios.count()

49

In [93]:
perfiles_df.groupby('Ciudad').sum().ix['3102064601']

CODBAR                     88524096
Sucursal                        TUN
Tratamiento                    sra.
Barrio                     la maria
CATEG                           CTA
Ocupación1               estudiante
Nivel Estudio1      básica primaria
Área de Estudio1                   
Name: 3102064601, dtype: object

In [94]:
ciudad_barrios

{'a': {},
 'acacias': {},
 'acacias meta': {'dorado bajo': 2},
 'agua de dios': {'san vicente': 2},
 'aipe': {'aipe': 3,
  'centro': 2,
  'jardin': 2,
  'los alpes': 2,
  'pueblo nuevo': 2,
  'san isidro': 3},
 'aldana': {},
 'algeciras': {},
 'ambalema tolima': {},
 'anapoima': {},
 'anserma caldas': {},
 'arcabuco': {},
 'arjona': {},
 'arjona bolivar': {},
 'armenia': {'7 de agosto': 2,
  'armenia': 3,
  'av. centenario': 2,
  'barrio galan': 2,
  'bosques de palermo': 2,
  'bosques de pinares': 3,
  'bosques de viena': 4,
  'caminos del campo': 4,
  'castellana': 6,
  'centro': 13,
  'ciudad dorada': 3,
  'ciudad real': 2,
  'coinca': 6,
  'condominio navarra': 2,
  'corbones': 4,
  'el caimo': 2,
  'el recreo': 2,
  'el retiro': 2,
  'el silencio': 3,
  'fundadores': 3,
  'galicia': 2,
  'genesis': 2,
  'granada': 5,
  'guayaquil': 3,
  'la aurora': 2,
  'la cabaña': 2,
  'la castellana': 7,
  'la clarita': 2,
  'la fachada': 4,
  'la fogata': 2,
  'la mariela': 3,
  'la miranda':

In [62]:
list(b.values).count('bogota')

2

In [59]:
b.value_counts()

                                  42
bogota                             2
san martin                         2
fontibon                           2
d                                  1
eduardo burgos                     1
bosa                               1
cuchilla del salado                1
mosquera                           1
san andres isla, colombia          1
edificio toscana                   1
parque industrial                  1
yesquita                           1
barrio nuevo                       1
popayan                            1
los amigos                         1
obapo                              1
la reserva del palmar              1
providencia                        1
pensionado                         1
enc.marcela i. hernandez y orl     1
colinas                            1
villa oriente                      1
alamos norte                       1
vereda san rafael                  1
tel.3421111 ext.5672               1
san mateo                          1
s

In [21]:
qqq=perfiles_df.groupby('Barrio').count()

In [26]:
qqq[qqq['CODBAR'] > 0].count()

CODBAR              11332
Sucursal            11332
Tratamiento         11332
Ciudad              11332
CATEG               11332
Ocupación1          11332
Nivel Estudio1      11332
Área de Estudio1    11332
Fech. Nacim         11332
Edad                11332
dtype: int64

In [25]:
qqq[qqq['CODBAR'] > 1].count()

CODBAR              3866
Sucursal            3866
Tratamiento         3866
Ciudad              3866
CATEG               3866
Ocupación1          3866
Nivel Estudio1      3866
Área de Estudio1    3866
Fech. Nacim         3866
Edad                3866
dtype: int64

In [42]:
qqq[qqq['CODBAR'] == 1].count()

CODBAR              7466
Sucursal            7466
Tratamiento         7466
Ciudad              7466
CATEG               7466
Ocupación1          7466
Nivel Estudio1      7466
Área de Estudio1    7466
Fech. Nacim         7466
Edad                7466
dtype: int64

In [9]:
%%time

sucursal_ciudades = {}

for index, row in perfiles_df.iterrows():
    sucursal = row['Sucursal']
    if sucursal:
        ciudad = row['Ciudad']
        sucursal_ciudades.setdefault(sucursal, defaultdict(int))
        sucursal_ciudades[sucursal][ciudad] += 1

CPU times: user 6.83 s, sys: 0 ns, total: 6.83 s
Wall time: 6.82 s


In [10]:
sucursal_ciudad_frecuencia = {}

for k in sucursal_ciudades:
    sucursal_ciudad_frecuencia[k] = Counter(sucursal_ciudades[k]).most_common(1)[0]

In [11]:
ciudades = Counter(dict(sucursal_ciudad_frecuencia.values()))

In [12]:
ciudades

Counter({'armenia': 454,
         'barranquilla': 228,
         'bogota': 22242,
         'bucaramanga': 344,
         'buenaventura': 317,
         'cali': 247,
         'cartagena': 707,
         'cucuta': 199,
         'florencia': 586,
         'girardot': 899,
         'honda': 334,
         'ibague': 2220,
         'ipiales': 983,
         'leticia': 477,
         'manizales': 788,
         'medellin': 163,
         'monteria': 177,
         'neiva': 1742,
         'pasto': 1674,
         'pereira': 1443,
         'popayan': 1841,
         'quibdo': 445,
         'riohacha': 237,
         'san andres': 658,
         'santa marta': 412,
         'sincelejo': 396,
         'tunja': 2930,
         'valledupar': 628,
         'villavicencio': 269})

In [ ]:
WORDS = ciudades

In [ ]:
%%time

def ciudad_corrector(x):
    ciudad = x['Ciudad']
    if ciudad:
        if ciudad in ciudades:
            r = ciudad
        else:
            words = re.findall(r'\w+', ciudad)
            candidates = set(w for w in words if w in ciudades)
            if len(candidates) == 1:
                r = candidates.pop()
            else:
                # len(candidates) > 1 no se da
                corrections = [correction(w) for w in words]
                candidates = set(c for c in corrections if c in ciudades)
                if len(candidates) == 1:
                    r = candidates.pop()
                else:
                    # len(candidates) > 1 sólo se da una vez pero las correcciones son incorrectas
                    r = ciudad
    else:
        r = ciudad
    return r

perfiles_df['Ciudad'] = perfiles_df.apply(ciudad_corrector, axis=1)

In [ ]:
perfiles_df['Ciudad'].nunique()

In [89]:
groups_df['Ciudad'].value_counts().head(40)

bogota              23087
tunja                2952
ibague               2264
popayan              1862
neiva                1754
pasto                1699
pereira              1455
ipiales               991
girardot              905
manizales             812
cartagena             721
san andres            664
valledupar            635
florencia             592
leticia               481
armenia               466
quibdo                446
santa marta           418
sincelejo             403
bucaramanga           371
pas                   365
dosquebradas          356
honda                 337
buenaventura          324
soacha                311
manizales caldas      309
villavicencio         292
cali                  269
riohacha              239
barranquilla          239
medellin              210
cucuta                207
monteria              185
pop                   145
chia                  136
                      108
floridablanca          84
bosa                   81
flandes     

In [12]:
groups_df[groups_df['Ciudad'] == '']['Ciudad'].count()

16

In [13]:
groups_df.shape[0] - groups_df[groups_df['Ciudad'] == '']['Ciudad'].count()

50256

In [86]:
groups_df = groups_df[groups_df['Ciudad'] != '']

In [96]:
groups_df.shape

(44966, 6)

In [94]:
%%time

c = perfiles_df['Ciudad'].value_counts()

def group_ciudad(x):
    e = x['Ciudad']
    if c[e] >= 100:
        r = e
    else:
        r = 'Otro'
    return r

groups_df['Ciudad'] = perfiles_df.apply(group_ciudad, axis=1)

CPU times: user 1.89 s, sys: 0 ns, total: 1.89 s
Wall time: 1.89 s


In [ ]:
groups_df['Ciudad'].value_counts()

#### Tratamiento

In [17]:
groups_df['Tratamiento'].nunique()

5

In [18]:
groups_df['Tratamiento'].value_counts()

sr.          22502
sra.         22339
              4984
otra inf.      392
inst            39
Name: Tratamiento, dtype: int64

In [19]:
groups_df[groups_df['Tratamiento'] == '']['Tratamiento'].count()

4984

In [20]:
groups_df.shape[0] - groups_df[groups_df['Tratamiento'] == '']['Tratamiento'].count()

45272

In [21]:
groups_df = groups_df[groups_df['Tratamiento'] != '']

In [22]:
groups_df.shape

(45272, 9)

In [23]:
%%time

c = perfiles_df['Tratamiento'].value_counts()

def group_tratamiento(x):
    e = x['Tratamiento']
    try:
        if e == 'sr.' or e == 'sra.':
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['Tratamiento'] = perfiles_df.apply(group_tratamiento, axis=1)

CPU times: user 1.31 s, sys: 7.15 ms, total: 1.32 s
Wall time: 1.31 s


In [24]:
groups_df['Tratamiento'].value_counts()

sr.     22502
sra.    22339
Otro      431
Name: Tratamiento, dtype: int64

#### Barrio

In [25]:
groups_df['Barrio'].nunique()

10387

In [35]:
groups_df['Barrio'].value_counts()

centro                             1034
chapinero                           403
                                    258
castilla                            225
cedritos                            220
kennedy                             215
fontibon                            214
modelia                             200
palermo                             199
san luis                            192
las nieves                          179
la soledad                          177
santa isabel                        175
santa ines                          163
el recuerdo                         162
teusaquillo                         154
san antonio                         151
santa barbara                       148
gaitan                              147
obrero                              146
ciudad salitre                      140
candelaria                          138
centenario                          131
san francisco                       130
suba                                128


In [40]:
'barrio de la esmeralda' in groups_df['Barrio'].unique()

False

In [43]:
for x in groups_df['Barrio'].unique():
    if 'barrio' in x:
        print(x)

barrio villa mayor
barrio palermo
campo real- barrio buenos aires
colombia/barrios unidos
barrio san diego
barrio espa a
barrio pie de la popa
san fernando/barrios unidos
12 de octubre/barrios unidos
los andes/barrios unidos
barrio 7 de agosto
los alcazares/barrios unidos
barrio dangond
barrio gaitan
barrio portal de alcazares
barrio chipre
barrio abajo
barrio ciudad valencia
barrio jose maria hernandez
barrio aurora
barrio kennedy
barrio laureles
barrio galan
barrio las acacias
barrio back road
barrio bight m17 c6
barrio la castilla
barrio alamos
barrio el centenario
barrio bitar
barrio san andres
san felipe/barrios unidos
barrio florencia
barrio nuevo
barrio colombia
barrio america
barrio la esmeralda
barrio champanag
fontibon barrio santander
barrio diana carolina
barrio san benito
barrio garupal etapa iii
barrios unidos
barrio 11 de noviembre
barrio obapo via pacurita
barrio la grecia
barrio los amigos
barrio mutis
barrio bajo
barrio obapo
barrio manuela beltran
barrio obrero
santa

In [29]:
from collections import Counter
c = Counter(groups_df['Barrio'].value_counts())

In [34]:
l=range(1,100)
ss1 = ss2 = 0
for k in c:
    if k not in l:
        ss1 += k * c[k]
    else:
        ss2 += k * c[k]
        
print(ss1)
print(ss2)
ss1+ss2

7842
37430


45272

In [51]:
c

Counter({1: 6818,
         2: 1150,
         3: 565,
         4: 318,
         5: 215,
         6: 174,
         7: 143,
         8: 83,
         9: 83,
         10: 63,
         11: 58,
         12: 44,
         13: 46,
         14: 39,
         15: 34,
         16: 25,
         17: 34,
         18: 32,
         19: 21,
         20: 23,
         21: 18,
         22: 24,
         23: 22,
         24: 14,
         25: 15,
         26: 17,
         27: 17,
         28: 10,
         29: 17,
         30: 16,
         31: 10,
         32: 7,
         33: 7,
         34: 7,
         35: 9,
         36: 8,
         37: 11,
         38: 5,
         39: 9,
         40: 3,
         41: 7,
         42: 4,
         43: 3,
         44: 5,
         45: 3,
         46: 6,
         47: 1,
         48: 2,
         49: 9,
         50: 3,
         51: 3,
         52: 3,
         53: 4,
         54: 4,
         55: 2,
         56: 2,
         57: 2,
         58: 2,
         59: 2,
         60: 2,
        

In [12]:
groups_df[groups_df['Barrio'] == '']['Barrio'].count()

16

In [13]:
groups_df.shape[0] - groups_df[groups_df['Barrio'] == '']['Barrio'].count()

50256

In [27]:
groups_df = groups_df[groups_df['Barrio'] != '']

In [28]:
groups_df.shape

(45014, 9)

In [29]:
%%time

c = perfiles_df['Barrio'].value_counts()

def group_barrio(x):
    e = x['Barrio']
    if c[e] >= 100:
        r = e
    else:
        r = 'Otro'
    return r

groups_df['Barrio'] = perfiles_df.apply(group_barrio, axis=1)

CPU times: user 2.03 s, sys: 8 ms, total: 2.04 s
Wall time: 2.03 s


In [30]:
groups_df['Barrio'].value_counts()

Otro                     35980
centro                    1034
chapinero                  403
castilla                   225
cedritos                   220
kennedy                    215
fontibon                   214
modelia                    200
palermo                    199
san luis                   192
las nieves                 179
la soledad                 177
santa isabel               175
santa ines                 163
el recuerdo                162
teusaquillo                154
san antonio                151
santa barbara              148
gaitan                     147
obrero                     146
ciudad salitre             140
candelaria                 138
centenario                 131
san francisco              130
suba                       128
santa lucia                127
galerias                   126
las americas               124
ciudad jardin              124
san vicente                120
                         ...  
alfonso lopez              119
chapiner

#### CATEG

In [96]:
groups_df['CATEG'].nunique()

16

In [101]:
groups_df['CATEG'].value_counts()

CTU    17098
CIS    15759
CTA    10666
CTB     1853
CEM     1602
CTF     1121
CTI     1018
CIP      339
CTC      302
CE1      216
CIN      130
CSU       92
CIV       68
CE2        6
CML        2
           2
Name: CATEG, dtype: int64

In [12]:
groups_df[groups_df['CATEG'] == '']['CATEG'].count()

16

In [13]:
groups_df.shape[0] - groups_df[groups_df['CATEG'] == '']['CATEG'].count()

50256

In [97]:
groups_df = groups_df[groups_df['CATEG'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df['CATEG'].value_counts()

def group_sucursal(x):
    e = x['CATEG']
    if c[e] >= 1000:
        r = e
    else:
        r = 'Otro'
    return r

groups_df['CATEG'] = perfiles_df.apply(group_sucursal, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


In [ ]:
groups_df['CATEG'].value_counts()

#### Ocupación1

In [106]:
groups_df['Ocupación1'].nunique()

30

In [107]:
groups_df['Ocupación1'].value_counts()

estudiante                 35805
empleado                    3908
docente                     2946
independiente               2549
                            1801
trabajador y estudiante     1016
pensionado                   920
hogar                        507
investigador                 258
desempleado                  181
universidad                  142
colegios                      91
cultural                      33
salud                         30
comercio                      28
instuto técnico               15
agropecuario                  12
instuto tecnológico            5
editorial                      5
alimentos                      3
financiero                     3
congregación religiosa         3
transporte                     2
construcción                   2
turismo                        2
telecomunicaciones             2
plástico                       2
minero y energético            1
bebidas                        1
automotriz                     1
Name: Ocup

In [12]:
groups_df[groups_df['Ocupación1'] == '']['Ocupación1'].count()

16

In [13]:
groups_df.shape[0] - groups_df[groups_df['Ocupación1'] == '']['Ocupación1'].count()

50256

In [97]:
groups_df = groups_df[groups_df['Ocupación1'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df['Ocupación1'].value_counts()

def group_ocupacion(x):
    e = x['Ocupación1']
    try:
        if e and c[e] >= 100:
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['Ocupación1'] = perfiles_df.apply(group_ocupacion, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


In [ ]:
groups_df['Ocupación1'].value_counts()

#### Nivel Estudio1

In [105]:
groups_df['Nivel Estudio1'].nunique()

13

In [106]:
groups_df['Nivel Estudio1'].value_counts()

universitario 1 nivel    9657
universitario 2 nivel    8121
profesional              6882
secundaria               6875
básica primaria          6660
                         6377
maestría                 2180
postgrado                1345
técnico                   711
graduado                  486
doctorado                 437
tecnológico               335
ninguno                   208
Name: Nivel Estudio1, dtype: int64

In [12]:
groups_df[groups_df['Nivel Estudio1'] == '']['Nivel Estudio1'].count()

16

In [13]:
groups_df.shape[0] - groups_df[groups_df['Nivel Estudio1'] == '']['Nivel Estudio1'].count()

50256

In [97]:
groups_df = groups_df[groups_df['Nivel Estudio1'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df['Nivel Estudio1'].value_counts()

def group_nivel_estudio(x):
    e = x['Nivel Estudio1']
    try:
        if c[e] >= 0:
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['Nivel Estudio1'] = perfiles_df.apply(group_nivel_estudio, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


In [ ]:
groups_df['Nivel Estudio1'].value_counts()

#### Área de Estudio1

In [96]:
groups_df['Área de Estudio1'].nunique()

16

In [98]:
groups_df['Área de Estudio1'].value_counts()

CTU    17098
CIS    15759
CTA    10666
CTB     1853
CEM     1602
CTF     1121
CTI     1018
CIP      339
CTC      302
CE1      216
CIN      130
CSU       92
CIV       68
CE2        6
CML        2
           2
Name: CATEG, dtype: int64

In [12]:
groups_df[groups_df['Área de Estudio1'] == '']['Área de Estudio1'].count()

16

In [13]:
groups_df.shape[0] - groups_df[groups_df['Área de Estudio1'] == '']['Área de Estudio1'].count()

50256

In [97]:
groups_df = groups_df[groups_df['Área de Estudio1'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df['Área de Estudio1'].value_counts()

def group_area_estudio(x):
    e = x['Área de Estudio1']
    try:
        if c[e] >= 1000:
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['Área de Estudio1'] = perfiles_df.apply(group_area_estudio, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


In [52]:
perfiles_df['Área de Estudio1'].value_counts()

50274

#### Edad

In [10]:
%%time

def bins_to_labels(bins):
    labels = []
    labels.append('{}-'.format(bins[1]))
    for i in range(1, len(bins) - 2):
        labels.append('{}-{}'.format(bins[i]+1, bins[i+1]))
    labels.append('{}+'.format(bins[-2]))
    return labels

bins = [-1, 11, 17, 25, 65, 100]
bins = [-1, 15,28,40,50,65, 100]
labels = bins_to_labels(bins)

def calculate_rango(x):
    pd.cut(pd.Series(range(101)), bins=bins, labels=labels)

perfiles_df['Edad'] = pd.cut(perfiles_df['Edad'], bins=bins, labels=labels)

TypeError: unorderable types: str() > float()

In [96]:
perfiles_df['Edad'].nunique()

16

In [98]:
perfiles_df['Edad'].value_counts()

CTU    17098
CIS    15759
CTA    10666
CTB     1853
CEM     1602
CTF     1121
CTI     1018
CIP      339
CTC      302
CE1      216
CIN      130
CSU       92
CIV       68
CE2        6
CML        2
           2
Name: CATEG, dtype: int64

In [97]:
groups_df['Edad'] = perfiles_df['Edad']
groups_df = groups_df[groups_df['Edad'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df[Edad'].value_counts()

def group_fecha_nacimiento(x):
    e = x['Edad']
    try:
        if c[e] >= 1000:
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['Edad'] = perfiles_df.apply(group_edad, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


In [52]:
perfiles_df['Edad'].count()

50274

In [100]:
groups_df.head()

,CODBAR,Sucursal,Tratamiento,Ciudad,CATEG
1,88183608,BLAA,sra.,bogota,CEM
2,88181732,BLAA,sra.,bogota,CEM
3,88182347,Otra,sr.,Otro,CEM
4,88181284,BLAA,sr.,bogota,CEM
6,88183786,BLAA,sr.,bogota,CEM


In [50]:
for e, c in perfiles_df['Sucursal'].value_counts().iteritems():
    if c >= 1000:
        groups_df[]

('BLAA  ', 24278)


#### Tratamiento

In [29]:
perfiles_df['Tratamiento'].nunique()

5

In [30]:
perfiles_df['Tratamiento'].value_counts()

sr.          22505
sra.         22341
              4997
otra inf.      392
inst            39
Name: Tratamiento, dtype: int64

In [ ]:
%%time

def bins_to_labels(bins):
    labels = []
    labels.append('{}-'.format(bins[1]))
    for i in range(1, len(bins) - 2):
        labels.append('{}-{}'.format(bins[i]+1, bins[i+1]))
    labels.append('{}+'.format(bins[-2]))
    return labels

bins = [-1, 11, 17, 25, 65, 1000]
bins = [-1, 15,28,40,50,65, 1000]
labels = bins_to_labels(bins)

def calculate_rango(x):
    pd.cut(pd.Series(range(101)), bins=bins, labels=labels)

perfiles_df['Rango'] = pd.cut(perfiles_df['Edad'], bins=bins, labels=labels)

In [ ]:
perfiles_df['Rango'].value_counts()

In [ ]:
perfiles_df['Edad'].value_counts()